In [1]:
import pandas as pd
import numpy as np
import requests

### Create streaming dataframe

In [2]:
# read your 1+ StreamingHistory files (depending on how extensive your streaming history is) into pandas dataframes
df_stream = pd.read_json('./MyData/StreamingHistory0.json')
#df_stream1 = pd.read_json('\MyData\streamingHistory1.json')

# merge streaming dataframes
#df_stream = pd.concat([df_stream0, df_stream1])

# create a 'UniqueID' for each song by combining the fields 'artistName' and 'trackName'
df_stream['UniqueID'] = df_stream['artistName'] + ":" + df_stream['trackName']

df_stream.head()

,endTime,artistName,trackName,msPlayed,UniqueID
0,2021-03-15 14:04,Crocodiles,Groove Is in the Heart / California Girls,682,Crocodiles:Groove Is in the Heart / California...
1,2021-03-15 15:09,Global News Podcast,Myanmar: Protesters killed in clashes,5674,Global News Podcast:Myanmar: Protesters killed...
2,2021-03-15 15:09,Conor Oberst,A Little Uncanny,124778,Conor Oberst:A Little Uncanny
3,2021-03-15 15:50,Global News Podcast,Myanmar: Protesters killed in clashes,1622749,Global News Podcast:Myanmar: Protesters killed...
4,2021-03-15 16:21,Global News Podcast,Myanmar: Protesters killed in clashes,9813,Global News Podcast:Myanmar: Protesters killed...


In [3]:
import json
from pandas.io.json import json_normalize


### Create library Dataframe

In [4]:

with open('./MyData/YourLibrary.json') as json_data:
    data = json.load(json_data)

df_library = pd.DataFrame(data['tracks'])


In [5]:

# add UniqueID column (same as above)
df_library['UniqueID'] = df_library['artist'] + ":" + df_library['track']

# add column with track URI stripped of 'spotify:track:'
new = df_library["uri"].str.split(":", expand = True)
df_library['track_uri'] = new[2]

df_library.head()

,artist,album,track,uri,UniqueID,track_uri
0,Johannes Brahms,Brahms: Hungarian Dances Nos. 1-21,"21 Hungarian Dances, WoO 1 (version for orches...",spotify:track:1e7uZfAoweDFcfrYVuifyk,"Johannes Brahms:21 Hungarian Dances, WoO 1 (ve...",1e7uZfAoweDFcfrYVuifyk
1,Jason Isbell,Southeastern,Live Oak,spotify:track:51JJuDBT4n2bgVasYgkBjD,Jason Isbell:Live Oak,51JJuDBT4n2bgVasYgkBjD
2,Steve Poltz,Live at the Belly Up,Kickin It Old School (Live),spotify:track:0mqcMV0qClE0p5hY7HqlT6,Steve Poltz:Kickin It Old School (Live),0mqcMV0qClE0p5hY7HqlT6
3,Chris Smither,Call Me Lucky,Change Your Mind,spotify:track:4tuWzu4mEMA2p0PPekTCpH,Chris Smither:Change Your Mind,4tuWzu4mEMA2p0PPekTCpH
4,Melody Pool,Deep Dark Savage Heart,Southern Nightshade,spotify:track:3nsW6nibHc3BtTblWF5EE0,Melody Pool:Southern Nightshade,3nsW6nibHc3BtTblWF5EE0


In [6]:
# create final dict as a copy df_stream
df_tableau = df_stream.copy()

# add column checking if streamed song is in library
# not used in this project but could be helpful for cool visualizations
df_tableau['In Library'] = np.where(df_tableau['UniqueID'].isin(df_library['UniqueID'].tolist()),1,0)

# left join with df_library on UniqueID to bring in album and track_uri
df_tableau = pd.merge(df_tableau, df_library[['album','UniqueID','track_uri']],how='left',on=['UniqueID'])

df_tableau.head()

,endTime,artistName,trackName,msPlayed,UniqueID,In Library,album,track_uri
0,2021-03-15 14:04,Crocodiles,Groove Is in the Heart / California Girls,682,Crocodiles:Groove Is in the Heart / California...,0,NaN,NaN
1,2021-03-15 15:09,Global News Podcast,Myanmar: Protesters killed in clashes,5674,Global News Podcast:Myanmar: Protesters killed...,0,NaN,NaN
2,2021-03-15 15:09,Conor Oberst,A Little Uncanny,124778,Conor Oberst:A Little Uncanny,0,NaN,NaN
3,2021-03-15 15:50,Global News Podcast,Myanmar: Protesters killed in clashes,1622749,Global News Podcast:Myanmar: Protesters killed...,0,NaN,NaN
4,2021-03-15 16:21,Global News Podcast,Myanmar: Protesters killed in clashes,9813,Global News Podcast:Myanmar: Protesters killed...,0,NaN,NaN


### Create Genre Dataframe

In [7]:
from env import CLIENT_ID, CLIENT_SECRET


In [8]:
# generate access token

# authentication URL
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [9]:
# used for authenticating all API calls
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [10]:
# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

In [11]:
# create blank dictionary to store track URI, artist URI, and genres
dict_genre = {}

# convert track_uri column to an iterable list
track_uris = df_library['track_uri'].to_list()

# loop through track URIs and pull artist URI using the API,
# then use artist URI to pull genres associated with that artist
# store all these in a dictionary
for t_uri in track_uris:
    
    dict_genre[t_uri] = {'artist_uri': "", "genres":[]}
    
    r = requests.get(BASE_URL + 'tracks/' + t_uri, headers=headers)
    r = r.json()
    a_uri = r['artists'][0]['uri'].split(':')[2]
    dict_genre[t_uri]['artist_uri'] = a_uri
    
    s = requests.get(BASE_URL + 'artists/' + a_uri, headers=headers)
    s = s.json()
    dict_genre[t_uri]['genres'] = s['genres']

KeyboardInterrupt: 

In [ ]:
# convert dictionary into dataframe with track_uri as the first column
df_genre = pd.DataFrame.from_dict(dict_genre, orient='index')
df_genre.insert(0, 'track_uri', df_genre.index)
df_genre.reset_index(inplace=True, drop=True)

df_genre.head()

In [ ]:
df_genre_expanded = df_genre.explode('genres')
df_genre_expanded.head()

In [ ]:
# save df_tableau and df_genre_expanded as csv files that we can load into Tableau
df_tableau.to_csv('MySpotifyDataTable.csv')
df_genre_expanded.to_csv('GenresExpandedTable.csv')

print('done')

---

In [13]:
pd.set_option('max_colwidth', 400)

In [12]:
#Bring in wranGgled data
df_tableau = pd.read_csv('MySpotifyDataTable.csv')
df_genre_expanded = pd.read_csv('GenresExpandedTable.csv')


In [15]:
df_tableau.head()

,Unnamed: 0,endTime,artistName,trackName,msPlayed,UniqueID,In Library,album,track_uri
0,0,2021-03-15 14:04,Crocodiles,Groove Is in the Heart / California Girls,682,Crocodiles:Groove Is in the Heart / California Girls,0,NaN,NaN
1,1,2021-03-15 15:09,Global News Podcast,Myanmar: Protesters killed in clashes,5674,Global News Podcast:Myanmar: Protesters killed in clashes,0,NaN,NaN
2,2,2021-03-15 15:09,Conor Oberst,A Little Uncanny,124778,Conor Oberst:A Little Uncanny,0,NaN,NaN
3,3,2021-03-15 15:50,Global News Podcast,Myanmar: Protesters killed in clashes,1622749,Global News Podcast:Myanmar: Protesters killed in clashes,0,NaN,NaN
4,4,2021-03-15 16:21,Global News Podcast,Myanmar: Protesters killed in clashes,9813,Global News Podcast:Myanmar: Protesters killed in clashes,0,NaN,NaN


In [16]:
df_genre_expanded.head()

,Unnamed: 0,track_uri,artist_uri,genres
0,0,1e7uZfAoweDFcfrYVuifyk,5wTAi7QkpP6kp8a54lmTOq,classical
1,0,1e7uZfAoweDFcfrYVuifyk,5wTAi7QkpP6kp8a54lmTOq,german romanticism
2,0,1e7uZfAoweDFcfrYVuifyk,5wTAi7QkpP6kp8a54lmTOq,late romantic era
3,1,51JJuDBT4n2bgVasYgkBjD,3Q8wgwyVVv0z4UEh1HB0KY,alternative country
4,1,51JJuDBT4n2bgVasYgkBjD,3Q8wgwyVVv0z4UEh1HB0KY,folk


---

### Get rid of podcasts 

When looking at the viz, podcasts are disproportionately represented by their length. Find a feature to filter them out and re-export